<a href="https://colab.research.google.com/github/uche-madu/huggingface_nlp_projects/blob/main/hf_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 5.5 MB/s eta 0:00:00


In [2]:
!pip install transformers datasets evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.3 MB/s eta 0:00:00


In [17]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    DataCollatorWithPadding,
    AdamWeightDecay
    )
from transformers.keras_callbacks import PushToHubCallback
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (TensorBoard, EarlyStopping)
import evaluate

import os
import datetime
import numpy as np

# Load the TensorBoard notebook extension
%load_ext tensorboard


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [6]:
raw_dataset = load_dataset("glue", "sst2")
raw_dataset

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

# Dataset class labels

In [7]:
label_names = raw_dataset["train"].features["label"].names
label_names

['negative', 'positive']

# Let us view some dataset examples

In [8]:
raw_dataset["train"][389]

{'sentence': "maybe it is formula filmmaking , but there 's nothing wrong with that if the film is well-crafted and this one is . ",
 'label': 1,
 'idx': 389}

In [9]:
raw_dataset["train"][29]

{'sentence': 'the plot is nothing but boilerplate clichés from start to finish , ',
 'label': 0,
 'idx': 29}

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

In [11]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [14]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_dataset["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# Finetuning the Model

In [15]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Set Hyperparameters

In [19]:
batch_size = 8
num_epochs = 15

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.

num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, 
    end_learning_rate=0.0, 
    decay_steps=num_train_steps,
    power=0.5
)

opt = Adam(learning_rate=lr_scheduler)

optimizer = AdamWeightDecay(5e-5, weight_decay_rate=0.01)

push_to_hub_callback = PushToHubCallback(
    "huggingface_model_output/", tokenizer=tokenizer, hub_model_id="bert-finetuned-sst2", 
)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

earlystopping_callback = EarlyStopping(
    monitor='val_accuracy', 
    patience=3,
    start_from_epoch=5,
    verbose=0
)

callbacks = [tensorboard_callback, earlystopping_callback, push_to_hub_callback]

/content/huggingface_model_output is already a clone of https://huggingface.co/dreemer6/bert-finetuned-sst2. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
model.compile(
    optimizer=opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


Freeze the bert block's weights to leverage on transfer learning

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 1,538
Non-trainable params: 109,482,240
_________________________________________________________________


Start Tensorboard before training to monitor progress

In [ ]:
%tensorboard --logdir logs

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs, callbacks=callbacks)

Epoch 1/15
8419/8419 [==============================] - 900s 101ms/step - loss: 0.2386 - accuracy: 0.9084 - val_loss: 0.2771 - val_accuracy: 0.8888
Epoch 2/15
1777/8419 [=====>........................] - ETA: 10:30 - loss: 0.1386 - accuracy: 0.9534

# Predictions on test data

In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]

In [ ]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

In [ ]:
preds, class_preds

In [ ]:
metric = evaluate.load("glue", "sst2")

In [ ]:
metric.compute(predictions=class_preds, references=raw_dataset["validation"]["label"])

In [ ]:
model.evaluate(tf_validation_dataset)

In [ ]:
model.push_to_hub("bert-finetuned-sst2", commit_message="End of training")

In [ ]:
model.config.id2label = {str(i): lbl for i, lbl in enumerate(label_names)}
model.config.label2id = {lbl: str(i) for i, lbl in enumerate(label_names)}

In [ ]:
repo_name = "bert-finetuned-sst2"
model.config.push_to_hub(repo_name)